In [1]:
# from statsmodels.genmod.families.links import Logit
import numpy as np
import pandas as pd
from tqdm import tqdm

import pickle
import joblib

from scipy.sparse import load_npz, csr_matrix

datasets = ['algebra05', 'assistments09', 'assistments15', 'assistments17', 'bridge_algebra06', 'spanish', 'statics'] # 'assistments12'

pd.set_option('display.max_rows', None)
# pd.reset_option('display.max_rows')

def loading(iters):
    return tqdm(iters, desc="Running ...", ascii=False, ncols=75)

def inv_logit(z):
  return 1 / (1 + np.exp(-z))

def calc_pfa_prob(s_opp, f_opp, s_slope, f_slope, stu_intercept, kc_intercept, ovr_intercept):
  z = (s_slope * s_opp) + (f_slope * f_opp) + stu_intercept + kc_intercept + ovr_intercept
  return inv_logit(z)

In [31]:
# import encode_fast as ef

# ACTIVE_FEATURES = ['i', 's', 'ic', 'sc', 'tc', 'w', 'a']
# def simulate_bestlr(ds):
#   csv_fp = f'./data/real/{ds}/preprocessed_data.csv'
#   model_fp = f'./data/real/{ds}/real-model.sav'
#   q_fp = f'./data/real/{ds}/q_mat.npz'

#   columns = ['user_id', 'item_id', 'timestamp', 'correct', 'skill_id', 's_opp', 'f_opp', 'prob']
#   opps = {}
#   rows = []
#   df = pd.read_csv(csv_fp, delimiter='\t')
#   Q_mat = load_npz(q_fp).toarray()

#   X = csr_matrix(load_npz(npz_fp))
#   X = X[:, 5:]
#   model = pickle.load(open(model_fp, 'rb'))
  
#   for idx, row in loading(list(df.iterrows())):
#     user_id = row['user_id']
#     skill_id = row['skill_id']

#     if user_id not in opps:
#       opps[user_id] = {}
#     if skill_id not in opps[user_id]:
#       opps[user_id][skill_id] = {'s': 0, 'f': 0}

#     prior_s = opps[user_id][skill_id]['s']
#     prior_f = opps[user_id][skill_id]['f']

#     dat = pd.DataFrame([[user_id, row['item_id'], row['timestamp'], None, skill_id, prior_s, prior_f, None]], columns=columns)

#     # user_id	item_id	timestamp	correct	skill_id
#     cur_df = df.Dataframe([], columns=df.columns)
#     X = ef.df_to_sparse(df, Q_mat, ACTIVE_FEATURES)
#     X = X[:, 5:]
#     prob = model.predict_proba(X[idx])[0][1]
#     correct = np.random.choice([1, 0], p=[prob, 1-prob])

#     opps[user_id][skill_id]['s' if correct == 1 else 'f'] += 1

#     dat['prob'] = prob
#     dat['correct'] = correct
#     rows.append(dat)
    
#   final = pd.concat(rows, axis=0)
#   final.to_csv(f'./simulation/simulated-data/{ds}.csv', index=False, sep='\t')

# # simulate_bestlr('./data/statics/preprocessed_data.csv', './data/statics/X-isicsctcwa.npz', './data/statics/real-model.sav', 'statics')
# # simulate('data/spanish/preprocessed_data.csv', 'spanish')

In [48]:
import encode_fast as ef
from importlib import reload
reload(ef)

ACTIVE_FEATURES = ['i', 's', 'ic', 'sc', 'tc', 'w', 'a']
def simulate_bestlr(ds):
  csv_fp = f'./data/real/{ds}/preprocessed_data.csv'
  model_fp = f'./data/real/{ds}/real-model.sav'
  q_fp = f'./data/real/{ds}/q_mat.npz'

  columns = ['user_id', 'item_id', 'timestamp', 'correct', 'skill_id', 's_opp', 'f_opp', 'prob']
  opps = {}
  rows = []
  df = pd.read_csv(csv_fp, delimiter='\t')
  Q_mat = load_npz(q_fp).toarray()

  model = pickle.load(open(model_fp, 'rb'))

  for user_id, group in loading(df.groupby('user_id')):
    if user_id not in opps:
      opps[user_id] = {}

    for idx in range(len(group)):
      row = group.iloc[idx]
      skill_id = row['skill_id']
      if skill_id not in opps[user_id]:
        opps[user_id][skill_id] = {'s': 0, 'f': 0}

      prior_s = opps[user_id][skill_id]['s']
      prior_f = opps[user_id][skill_id]['f']
      dat = pd.DataFrame([[user_id, row['item_id'], row['timestamp'], None, skill_id, prior_s, prior_f, None]], columns=columns)

      X = ef.df_to_sparse(group[:idx+1], Q_mat, ACTIVE_FEATURES)
      X = X[:, 5:]

      prob = model.predict_proba(X)[0][1]
      correct = np.random.choice([1, 0], p=[prob, 1-prob])

      df.loc[idx, 'correct'] = correct
      opps[user_id][skill_id]['s' if correct == 1 else 'f'] += 1

      dat['prob'] = prob
      dat['correct'] = correct
      rows.append(dat)

  final = pd.concat(rows, axis=0)
  final.to_csv(f'./simulation/simulated-data/{ds}/{ds}.csv', index=False, sep='\t')

simulate_bestlr('algebra05')
# simulate('data/spanish/preprocessed_data.csv', 'spanish')

Running ...:   0%|                                 | 0/567 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import os
import encode_fast as ef
# for f in os.listdir('./data'):
# for f in ['assistments09']:
for f in ['algebra05', 'assistments09', 'assistments15', 'assistments17', 'bridge_algebra06', 'spanish', 'statics', 'assistments12']
  print(f)
  simulate_bestlr(f'./data/{f}/preprocessed_data.csv', f'./data/{f}/real-model.sav', f)

# from scipy.sparse import load_npz, csr_matrix
# data_fp = "./data/statics/X-isicsctcwa.npz"
# X = csr_matrix(load_npz(data_fp))

In [2]:
import os

folder_path = "data"
folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

print(folders)

['algebra05', 'assistments09', 'assistments12', 'assistments15', 'assistments17', 'assistments17_first_attempt', 'assistments17_single_attempt', 'bridge_algebra06', 'spanish', 'statics']


In [1]:
import os
files = [f'./data/{f}/real-model.sav'for f in os.listdir('./data')]
for f in files:
  size = os.path.getsize(f)
  print(f"{f} - {size/(1024*1024)}")


./data/algebra05/real-model.sav - 1.3240928649902344
./data/assistments09/real-model.sav - 0.13869667053222656
./data/assistments12/real-model.sav - 0.41005897521972656
./data/assistments15/real-model.sav - 0.0038318634033203125
./data/assistments17/real-model.sav - 0.027238845825195312
./data/bridge_algebra06/real-model.sav - 0.9982643127441406
./data/spanish/real-model.sav - 0.008958816528320312
./data/statics/real-model.sav - 0.012353897094726562


What do I need to do?

- Fit BestLR and the use it to generate data + know the ground truth
- Try to fit Both BestLR, PFA, and DKT on the generated data


assisstments09, assistments15, spanish, algebra05

In [112]:
dataset = "statics"

def check_q_all_one(dataset):
  dir_path = f"./data/real/{dataset}"
  df = pd.read_csv(os.path.join(dir_path, "preprocessed_data.csv"), sep="\t")
  df = df[["user_id", "item_id", "timestamp", "correct", "skill_id"]]
  Q_mat = sparse.load_npz(os.path.join(dir_path, 'q_mat.npz')).toarray()

  if any([sum(q) != 1 for q in Q_mat]):
    print(f'{dataset}: There is != 1')
  else:
    print(f'{dataset}: Ok!')

# datasets = [f for f in os.listdir('./data/real')]
datasets = ['algebra05', 'assistments09', 'assistments15', 'assistments17', 'bridge_algebra06', 'spanish', 'statics', 'assistments12']
for d in datasets:
  check_q_all_one(d)

# check_q_all_one('algebra05')


algebra05: There is != 1
assistments09: There is != 1
assistments15: Ok!
assistments17: There is != 1
bridge_algebra06: There is != 1
spanish: Ok!
statics: Ok!
assistments12: Ok!


In [47]:
from importlib import reload
import encode_fast as ef
# import encode as ec
from scipy import sparse
import os, time

reload(ef)

dataset = "algebra05"

dir_path = f"./data/real/{dataset}"
df = pd.read_csv(os.path.join(dir_path, "preprocessed_data.csv"), sep="\t")
df = df[["user_id", "item_id", "timestamp", "correct", "skill_id"]]
Q_mat = sparse.load_npz(os.path.join(dir_path, 'q_mat.npz')).toarray()

# Transform q-matrix into dictionary for fast lookup
num_items, num_skills = Q_mat.shape
Q_mat_dict = {i: set() for i in range(num_items)}
for i, j in np.argwhere(Q_mat == 1):
    Q_mat_dict[i].add(j)

active_features = ['i', 's', 'ic', 'sc', 'tc', 'w', 'a']
print('start ...')
start = time.time()
X = ef.df_to_sparse(df, Q_mat, active_features)
print(X[:,5:])
print('---')
end = time.time()
print('Timespent: ', (end-start), 's')

# 3 seconds

start ...
  (0, 66)	1.0
  (0, 160001)	1.0
  (1, 66)	1.0
  (1, 178)	0.6931471805599453
  (1, 225)	0.6931471805599453
  (1, 292)	0.6931471805599453
  (1, 339)	0.6931471805599453
  (1, 160002)	1.0
  (2, 83)	1.0
  (2, 87)	1.0
  (2, 225)	1.0986122886681098
  (2, 339)	1.0986122886681098
  (2, 160005)	1.0
  (3, 30)	1.0
  (3, 72)	1.0
  (3, 101)	1.0
  (3, 225)	1.3862943611198906
  (3, 339)	1.3862943611198906
  (3, 160006)	1.0
  (4, 30)	1.0
  (4, 72)	1.0
  (4, 101)	1.0
  (4, 142)	0.6931471805599453
  (4, 184)	0.6931471805599453
  (4, 213)	0.6931471805599453
  :	:
  (1128, 20)	1.0
  (1128, 29)	1.0
  (1128, 42)	1.0
  (1128, 132)	4.382026634673881
  (1128, 141)	3.2188758248682006
  (1128, 154)	4.343805421853684
  (1128, 224)	0.6931471805599453
  (1128, 225)	7.029087564149662
  (1128, 246)	3.5263605246161616
  (1128, 255)	2.3978952727983707
  (1128, 268)	3.8066624897703196
  (1128, 339)	6.525029657843462
  (1128, 157966)	1.0
  (1129, 20)	1.0
  (1129, 132)	4.394449154672439
  (1129, 225)	7.0299729117

In [36]:
dataset = "statics"

dir_path = f"./data/real/{dataset}"
df = pd.read_csv(os.path.join(dir_path, "preprocessed_data.csv"), sep="\t")
print(sorted(df.item_id.unique()))
# df[df.item_id == 0]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [26]:
npz_fp = './data/real/statics/X-isicsctcwa.npz'
X = csr_matrix(load_npz(npz_fp))
X = X[:, 5:]
print(X.shape)

(189297, 1521)
